***Конвертация JSON2HTML***

Выполнил: Юрий Куськов y.kuskov@solarl.ru

---

In [1]:
import json

In [2]:
def read_json(file):
    try:
        f = open(file,'r')
    except IOError as e:
        print(u'не удалось открыть файл')
    else:
        with f:
            return json.loads(f.read())
    finally:
        f.close()

С функцией *json.loads()* имеются проблемы. Возможна ситуация с потерей порядка ключей при переносе из *json* в *dict* ?

---

**1 задание**
Формат: список (параграф:заголовок)

In [3]:
def func_1(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    return "".join(["<h1>{}</h1><p>{}</p>".format(x["title"],x["body"]) for x in read_json(file)])

In [4]:
func_1('1st/source.json')

'<h1>Title#1</h1><p>Hello,World1!</p><h1>Title#2</h1><p>Hello,World2!</p>'

---

**2 задание**
В ключ добавлено название тега

In [5]:
def func_2(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    _in = read_json(file)
    _out = ""
    for x in _in:                     # цикл по словарям
        for k in x:                   # цикл внутри словаря
            _out += "<{0}>{1}</{0}>".format(k,x[k])
    return _out

In [6]:
func_2('2nd/source.json')

'<h3>Title#1</h3><div>Hello,World1!</div>'

---

**3 задание**
Если *list* - то все элементы, которые содержатся - должны быть обернуты в *ul*, а каждый конкретный элемент в списке в тег *li*

In [7]:
def func_3(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    _in = read_json(file)
    if isinstance(_in,list):
        _out = ""
        for x in _in:
            _out +="<li>"
            for k in x:
                _out += "<{0}>{1}</{0}>".format(k,x[k])
            _out +="</li>"
        #_out += "</ul>"
        return "<ul>"+_out+"</ul>"
    else: raise ValueError("Объект не является списком")

In [8]:
func_3('3rd/source.json') 

'<ul><li><h3>Title#1</h3><div>Hello,World1!</div></li><li><h3>Title#2</h3><div>Hello,World2!</div></li></ul>'

---

**4 задание**
Теперь список может появиться в любом месте, а элементы могут быть вложены друг в друга

In [55]:
def treeHTML(s):  
    """функция рекурсивного анализа json"""
    if isinstance(s,list):
        return "<ul>"+"".join(["<li>"+treeHTML(d)+"</li>" for d in s]) +"</ul>" 
    if isinstance(s,dict):
        _out = ""
        for k in s:
            if isinstance(s[k],list):
                _out += "<{0}>{1}</{0}>".format(k,treeHTML(s[k]))
            else:
                _out += "<{0}>{1}</{0}>".format(k,s[k])
        return _out

In [56]:
def func_4(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    _in = read_json(file)
    return treeHTML(_in)

In [58]:
func_4('4th/source.json')

'<ul><li><span>Title#1</span><content><ul><li><p>Example1</p><header>header1</header></li></ul></content></li><li><div>div1</div></li></ul>'

---

**5 задание**
Верстка поплыла - необхоимо добавлять класс и идентификаторы к тегам, а содержимое не должно рассматриваться как html

In [23]:
def getTagClassesId(s):  # надо учесть что id может идти перед классами и переделать это...
    """получение тега, класса и айди из элемента словаря"""
    _tag, _class, _id = None, None, None
    n_class, n_id = s.count('.'), s.count('#')
    if n_id > 0:
        s, _id = s.split('#')
    if n_class > 0:
        s = s.split('.')
        _class = s[1:]
        s = s[0]
    _tag = s
    return _tag, " ".join(_class), _id

In [44]:
from html import escape   # замена символов на их соответсвующий html-код

In [42]:
def func_5(file):
    """в параметре file указывается путь к файлу 'folder/file.json'"""
    _in = read_json(file)
    _out = ""
    for k in _in:
        _tag, _class, _id = getTagClassesId(k)
        if _class is not None and _id is not None:
            _out += '<{0} id="{2}" class="{1}">{3}</{0}>'.format(_tag, _class, _id, escape(_in[k]))
        elif _class is not None and _id is None:
            _out += '<{0} class="{1}">{2}</{0}>'.format(_tag, _class, escape(_in[k]))
        elif _class is None and _id is not None:
            _out += '<{0} id="{1}">{2}</{0}>'.format(_tag, _id, escape(_in[k]))
    return _out 

In [43]:
func_5('5th/source.json')

'<p id="my-id" class="my-class">hello</p><p class="my-class1 my-class2">example&lt;a&gt;asd&lt;/a&gt;</p>'